In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


For this problem I am going to be solving the problem of determining whether someone does or does not have diabetes based on their present symptoms and various attributes such as age and gender. It makes sense to use logistic regression in this case because the output is a binary positive or negative and we need to determine the probability of either outcome.

In [55]:
## Reading the dataset

import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Datasets/diabetes_risk_prediction_dataset.csv')
df.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [56]:
# determine the number of missing entries in each column
print(len(df) - df.count())

Age                   0
Gender                0
Polyuria              0
Polydipsia            0
sudden weight loss    0
weakness              0
Polyphagia            0
Genital thrush        0
visual blurring       0
Itching               0
Irritability          0
delayed healing       0
partial paresis       0
muscle stiffness      0
Alopecia              0
Obesity               0
class                 0
dtype: int64


In [57]:
# determine the distribution of positive to negative classifications

p = df['class'].value_counts()['Positive']
n = df['class'].value_counts()['Negative']

print(f"Positive Cases: {p}\nNegative Cases: {n}")

Positive Cases: 320
Negative Cases: 200


In [58]:
# find the number of unique values in each column do determine which column can use binary encoding, and in which column
# might benefit from one hot encoding

df.nunique()

Age                   51
Gender                 2
Polyuria               2
Polydipsia             2
sudden weight loss     2
weakness               2
Polyphagia             2
Genital thrush         2
visual blurring        2
Itching                2
Irritability           2
delayed healing        2
partial paresis        2
muscle stiffness       2
Alopecia               2
Obesity                2
class                  2
dtype: int64

In [59]:
# double cheeck the datatype for each column
df.dtypes

Age                    int64
Gender                object
Polyuria              object
Polydipsia            object
sudden weight loss    object
weakness              object
Polyphagia            object
Genital thrush        object
visual blurring       object
Itching               object
Irritability          object
delayed healing       object
partial paresis       object
muscle stiffness      object
Alopecia              object
Obesity               object
class                 object
dtype: object

In [60]:
# take note of what what Positive and Negative encode to after binary encoding

c = df['class'].astype('category')
d = dict(enumerate(c.cat.categories))
print(d)

{0: 'Negative', 1: 'Positive'}


In [61]:
# convert the labels in each column to integer values instead
columns = df.columns
binary_columns = df.columns[1:] #these are columns with only two unique values in them

for column_name in binary_columns:
  df[column_name] = df[column_name].astype('category').cat.codes

df.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1
1,58,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1
2,41,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1
3,45,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,1
4,60,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1


In [62]:
# shuffle the dataset just to ensure a more or less even distribution of positive to negative cases
df = df.sample(frac = 1).reset_index(drop=True) #reset_index mainintains a sequential index ordering
df.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,45,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0
1,28,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1
2,48,0,1,1,1,1,1,0,0,0,0,0,1,0,0,0,1
3,53,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0
4,35,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1


In [63]:
ratio = 0.8 #the proportion of the full dataset dedicated
rows = len(df)  #total number of rows in the full dataset

training_size = int(rows * ratio)

# Split data into testing and training data
training_data = df[0:training_size]
testing_data = df[training_size:]

training_data.head()

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,45,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0
1,28,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1
2,48,0,1,1,1,1,1,0,0,0,0,0,1,0,0,0,1
3,53,1,0,0,0,1,1,0,1,1,0,1,1,1,1,0,0
4,35,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1


In [64]:
#determine the distribution of positive to negative classifications in training data

p = training_data['class'].value_counts()[1]
n = training_data['class'].value_counts()[0]

print(f"Positive Cases: {p}\nNegative Cases: {n}")


Positive Cases: 252
Negative Cases: 164


After evaluating the data, we see that there are no missing values and that the majority of categories have only two types of labels in them with the exception of age. For this reason I did a binary encoding for most of the dataset and then a mean normalization on the age category. The normalization was to reduce instability in the model caused by large ranging values in only the age category.

In [65]:
# convert the training and testing data into tensors

import math
import numpy as np
import torch

X_train, y_train = torch.FloatTensor(df.drop(columns=['class']).values), torch.FloatTensor(df['class'].values).reshape(-1, 1)
X_test, y_test = torch.FloatTensor(df.drop(columns=['class']).values), torch.FloatTensor(df['class'].values).reshape(-1, 1)

print(f"X_train Shape: {X_train.shape},  y_train shape {y_train.shape}")
print(f"X_test Shape: {X_test.shape},  y_test shape {y_test.shape}")

X_train Shape: torch.Size([520, 16]),  y_train shape torch.Size([520, 1])
X_test Shape: torch.Size([520, 16]),  y_test shape torch.Size([520, 1])


In [66]:
def sigmoid(p):
  return 1/(1+torch.exp(-p))

In [67]:
def batchGradientDescent(X, y, w, b, theta, eta = 0.01):
  #initialize the weight and bias gradients
  w_grad = torch.rand(16, 1)
  b_grad = torch.rand(1, 1)

  #initialize the gradient to ones using
  grad = torch.cat([w_grad, b_grad], dim=0)
  grad_norm = float('inf') #initalize as inf for upper bound

  while(grad_norm > theta): #calculate the euclidean norm to determine if stopping criteria has been met
    # print(grad.norm(dim=0, p=2))

    # calculate the predictions
    y_preds = sigmoid(X @ w + b) #implement the sigmoid function

    # calculate the loss based on the predictions
    loss = -torch.mean((y * torch.log(y) + (1 - y_preds) * torch.log(1-y_preds)))

    #print("\rLOSS: {:.2f}".format(loss), end='', flush=True)

    # calculate the gradient of the loss function
    w_grad = X_train.T @ (y_preds - y_train)
    b_grad = torch.mean(y_preds - y_train)

    #update the gradient and recompute the Euclidean norm
    grad = torch.cat([w_grad, b_grad.reshape(1, 1)], dim = 0)
    grad_norm = grad.norm(dim=0, p=2)[0]

    #print("\rGRADIENT NORM: {:.2f}".format(grad_norm), end='', flush=True)

    # update the weights and biases based on the learning rate and gradient
    w -= eta * w_grad
    b -= eta * b_grad

  #print("\rFINAL LOSS: {:.2f}".format(loss))
  #print("\rGRADIENT NORM: {:.2f}".format(grad_norm))

  #look into smote

In [68]:
def minibatchGradientDescent(X, y, w, b, num_epochs = 10, batch_size = 32, eta = 0.01):
  #initialize the weight and bias gradients
  w_grad = torch.rand(16, 1)
  b_grad = torch.rand(1, 1)

  #initialize the gradient to ones using
  grad = torch.cat([w_grad, b_grad], dim=0)
  grad_norm = float('inf') #initalize as inf for upper bound

  for epoch_idx in range(num_epochs):
    epoch = X[torch.randperm(X.size(0))] #generate a random permutation of X rows
    total_loss = 0

    #print("EPOCH {}".format(epoch_idx + 1))
    for batch_idx in range(batch_size, X.size(0), batch_size):
      #generate the minibatch from the epoch
      minibatch = epoch[[batch_idx - batch_size, batch_idx]]

      #calculate the predictions
      y_preds = sigmoid(X @ w + b)

      #clamp to avoid 0 and 1 in the prediction set
      clamped_y_preds = torch.clamp(y_preds, 0.00001, 0.99999)

      # calculate the loss based on the predictions
      loss = -torch.mean((y * torch.log(clamped_y_preds) + (1 - y) * torch.log(1-clamped_y_preds)))
      total_loss += loss

      #print("\tLOSS: {:.2f}".format(loss))

      # calculate the gradient of the loss function
      w_grad = X_train.T @ (y_preds - y_train)
      b_grad = torch.mean(y_preds - y_train)

      #update the gradient and recompute the Euclidean norm
      grad = torch.cat([w_grad, b_grad.reshape(1, 1)], dim = 0)
      grad_norm = grad.norm(dim=0, p=2)[0]

      #print("\rGRADIENT NORM: {:.2f}".format(grad_norm), end='', flush=True)

      # update the weights and biases based on the learning rate and gradient
      w -= eta * w_grad
      b -= eta * b_grad

    #print("\n\tAVG. LOSS: {:.2f}".format(total_loss/(X.size(0) // batch_size))) #print average loss of the epoch
    #print("\rGRADIENT NORM: {:.2f}".format(grad_norm))

In [69]:
def stochasticGradientDescent(X, y, w, b, num_epochs = 10, eta = 0.01):
  #initialize the weight and bias gradients
  w_grad = torch.rand(16, 1)
  b_grad = torch.rand(1, 1)

  # initialize the gradient to ones using
  grad = torch.cat([w_grad, b_grad], dim=0)
  grad_norm = float('inf') #initalize as inf for upper bound

  for epoch_idx in range(num_epochs):
    epoch = X[torch.randperm(X.size(0))] #generate a random permutation of X rows
    total_loss = 0

    #print("EPOCH {}".format(epoch_idx + 1))
    for instance in epoch:
      # calculate the predictions
      y_preds = sigmoid(instance @ w + b)

      #clamp to avoid 0 and 1 in the prediction set
      clamped_y_preds = torch.clamp(y_preds, 0.00001, 0.99999)

      # calculate the loss based on the predictions
      loss = -torch.mean((y * torch.log(clamped_y_preds) + (1 - y) * torch.log(1-clamped_y_preds)))
      total_loss += loss

      #print("\rLOSS: {:.2f}".format(loss), end = '', flush = True)

      # calculate the gradient of the loss function
      w_grad = X_train.T @ (y_preds - y_train)
      b_grad = torch.mean(y_preds - y_train)

      #update the gradient and recompute the Euclidean norm
      grad = torch.cat([w_grad, b_grad.reshape(1, 1)], dim = 0)
      grad_norm = grad.norm(dim=0, p=2)[0]

      #print("\rGRADIENT NORM: {:.2f}".format(grad_norm), end='', flush=True)

      # update the weights and biases based on the learning rate and gradient
      w -= eta * w_grad
      b -= eta * b_grad

    #print("\tAVG. LOSS: {:.2f}".format(total_loss/X.size(0))) #print average loss of the epoch
    #print("\rGRADIENT NORM: {:.2f}".format(grad_norm), end = '', flush = True)

In [70]:
#store the number of features
num_features = X_train.size(dim=1)

#initalize the weights and bias for minibatch gradient descent method
batch_weights = torch.rand((num_features, 1))
batch_bias = torch.rand(1)

#initalize the weights and bias for minibatch gradient descent method
minibatch_weights = torch.rand((num_features, 1))
minibatch_bias = torch.rand(1)

#initalize the weights and bias for stochastic gradient descent method
stochastic_weights = torch.rand((num_features, 1))
stochastic_bias = torch.rand(1)

In [71]:
batchGradientDescent(X_train, y_train,
                     batch_weights, batch_bias,
                     theta = 11,
                     eta = 0.005)

In [72]:
minibatchGradientDescent(X_train, y_train,
                         minibatch_weights, minibatch_bias,
                         num_epochs = 300,
                         batch_size = 32,
                         eta = 0.005)

In [73]:
stochasticGradientDescent(X_train, y_train,
                         stochastic_weights, stochastic_bias,
                         num_epochs = 300,
                         eta = 0.005)

In [74]:
#measure the accuracy of the model using the testing set
batch_y_preds = sigmoid(X_test @ batch_weights + batch_bias)

batch_accuracy = 100 * torch.eq(y_test, batch_y_preds).sum()/y_test.size(0) #generates a tensor with True or False depending on if the values are equal
print("BATCH GADIENT DESCENT ACCURACY: {:.2f}%".format(batch_accuracy))

BATCH GADIENT DESCENT ACCURACY: 91.54%


In [75]:
#measure the accuracy of the model using the testing set
minibatch_y_preds = sigmoid(X_test @ minibatch_weights + minibatch_bias)

minibatch_accuracy = 100 * torch.eq(y_test, minibatch_y_preds).sum()/y_test.size(0)
print("MINIBATCH GADIENT DESCENT ACCURACY: {:.2f}%".format(minibatch_accuracy))

MINIBATCH GADIENT DESCENT ACCURACY: 89.23%


In [76]:
#measure the accuracy of the model using the testing set
stochastic_y_preds = sigmoid(X_test @ stochastic_weights + stochastic_bias)

stochastic_accuracy = 100 * torch.eq(y_test, stochastic_y_preds).sum()/y_test.size(0)
print("STOCHASTIC GADIENT DESCENT ACCURACY: {:.2f}%".format(stochastic_accuracy))

STOCHASTIC GADIENT DESCENT ACCURACY: 89.04%


In [77]:
def adam(X, y, w, b, num_epochs = 10, eta = 0.01, beta1 = 0.9, beta2 = 0.999, theta = 10, epsilon = 10e-8):
  #initialize the weight and bias gradients
  w_grad = torch.rand(16, 1)
  b_grad = torch.rand(1, 1)

  #initialize the gradient to ones using
  grad = torch.cat([w_grad, b_grad], dim=0)
  grad_norm = float('inf') #initalize as inf for upper bound

  #initialize the moment vectors
  M = torch.zeros(17, 1)
  V = torch.zeros(17, 1)

  #initialize index to track the current m and v
  t = 0

  for epoch_idx in range(num_epochs):
    epoch = X[torch.randperm(X.size(0))] #generate a random permutation of X rows
    total_loss = 0

    #print("EPOCH {}".format(epoch_idx + 1), flush = True)
    for instance in epoch:
      #iterate the index t
      t += 1

      # calculate the predictions
      y_preds = sigmoid(instance @ w + b)

      #clamp to avoid 0 and 1 in the prediction set
      clamped_y_preds = torch.clamp(y_preds, 0.00001, 0.99999)

      # calculate the loss based on the predictions
      loss = -torch.mean((y * torch.log(clamped_y_preds) + (1 - y) * torch.log(1-clamped_y_preds)))
      total_loss += loss

      #print("\rLOSS: {:.2f}".format(loss), end = '', flush = True)

      # calculate the gradient of the loss function
      w_grad = X_train.T @ (y_preds - y_train)
      b_grad = torch.mean(y_preds - y_train)

      #update the gradient and recompute the Euclidean norm
      grad = torch.cat([w_grad, b_grad.reshape(1, 1)], dim = 0)
      grad_norm = grad.norm(dim=0, p=2)[0]

      #print("\rGRADIENT NORM: {:.2f}".format(grad_norm), end='', flush=True)

      #terminate if there is approximate convergence based on theta
      if(grad_norm <= theta):
          return

      #update biased first moment estimate and second raw moment estimates
      m = beta1 * M[:, t-1].unsqueeze(1) + (1 - beta1) * grad
      v = beta2 * V[:, t-1].unsqueeze(1) + (1 - beta2) * torch.square(grad)

      M = torch.cat((M, m), dim=1)
      V = torch.cat((V, v), dim=1)

      #compute the bias-corrected first moment estimates, and second raw raw moment estimates
      m_hat = m/(1-(beta1**t))
      v_hat = v/(1-(beta2**t))

      # update the weights and biases based on the learning rate and gradient
      w -= eta * m_hat[:16, :]/(torch.sqrt(v_hat[:16, :]) + epsilon)
      b -= eta * m_hat[-1, :]/(torch.sqrt(v_hat[-1, :]) + epsilon)

    #print("\rGRADIENT NORM: {:.2f}".format(grad_norm), end = '', flush = True)

In [78]:
def adagrad(X, y, w, b, num_epochs = 10, eta = 0.01, theta = 10, epsilon = 10e-8):
  #initialize the weight and bias gradients
  w_grad = torch.zeros(16, 1)
  b_grad = torch.zeros(1, 1)

  #initialize the gradient to ones using
  grad = torch.cat([w_grad, b_grad], dim=0)
  grad_norm = float('inf') #initalize as inf for upper bound

  #initialize the moment vectors
  G = torch.zeros(17, 17)

  #initialize index to track the time step
  t = 0

  for epoch_idx in range(num_epochs):
    epoch = X[torch.randperm(X.size(0))] #generate a random permutation of X rows
    total_loss = 0

    print("\rEPOCH {}".format(epoch_idx + 1), end = '', flush = True)
    for instance in epoch:
      #iterate the index t
      t += 1

      # calculate the predictions
      y_preds = sigmoid(instance @ w + b)

      #clamp to avoid 0 and 1 in the prediction set
      clamped_y_preds = torch.clamp(y_preds, 0.00001, 0.99999)

      # calculate the loss based on the predictions
      loss = -torch.mean((y * torch.log(clamped_y_preds) + (1 - y) * torch.log(1-clamped_y_preds)))
      total_loss += loss

      #print("\rLOSS: {:.2f}".format(loss), end = '', flush = True)

      # calculate the gradient of the loss function
      w_grad = X_train.T @ (y_preds - y_train)
      b_grad = torch.mean(y_preds - y_train)

      #update the gradient and recompute the Euclidean norm
      grad = torch.cat([w_grad, b_grad.reshape(1, 1)], dim = 0)
      grad_norm = grad.norm(dim=0, p=2)[0]

      G += (grad @ grad.T) #recalculate the outerproduct sum G

      #print("\rGRADIENT NORM: {:.2f}".format(grad_norm), end='', flush=True)

      #terminate if there is approximate convergence based on theta
      if(grad_norm <= theta):
          return

      # update the weights and biases based on the learning rate and gradient
      step = eta * torch.rsqrt(G+epsilon) @ grad

      w -= step[:16, :]
      b -= step[-1, :]

    #print("\rGRADIENT NORM: {:.2f}".format(grad_norm), end = '', flush = True)

In [79]:
#initalize the weights and bias for adam optimized gradient descent method
adam_weights = torch.rand((num_features, 1))
adam_bias = torch.rand(1)

#initalize the weights and bias for adagrad optimized gradient descent method
adagrad_weights = torch.rand((num_features, 1))
adagrad_bias = torch.rand(1)

In [80]:
adam(X_train, y_train, adam_weights, adam_bias, num_epochs = 100, eta = 0.001, beta1 = 0.9, beta2 = 0.999, theta = 11, epsilon = 10e-8)

In [81]:
adagrad(X_train, y_train, adagrad_weights, adagrad_bias, num_epochs = 200, eta = 0.01, theta = 10, epsilon = 10e-8)

EPOCH 200

In [82]:
#measure the accuracy of the model using the testing set
adam_y_preds = sigmoid(X_test @ adam_weights + adam_bias)

adam_accuracy = 100 * torch.eq(y_test, adam_y_preds).sum()/y_test.size(0)
print("STOCHASTIC GADIENT DESCENT ACCURACY: {:.2f}%".format(adam_accuracy))

STOCHASTIC GADIENT DESCENT ACCURACY: 52.69%


In [83]:
#measure the accuracy of the model using the testing set
adagrad_y_preds = sigmoid(X_test @ adagrad_weights + adagrad_bias)

adagrad_accuracy = 100 * torch.eq(y_test, adagrad_y_preds).sum()/y_test.size(0)
print("ADAGRAD GADIENT DESCENT ACCURACY: {:.2f}%".format(adagrad_accuracy))

ADAGRAD GADIENT DESCENT ACCURACY: 3.27%


Based on the raw amount of correct responses, it seems the optimization did worse than the basic gradient descent. This is probably do the lack of data. While it may be possible to get better results using the optimizations, it may take much more time than would be necessary given the speed of the batch gradient descent. Accuracy was measured by comparing the raw number of correct responses as a percent of the testing data.
